### *새로 뽑은 뉴스 앞부분 뒷부분에 합치기 + 긍부정 점수 매긴 후 분류 작업*
#### train data에 해당되는 20.1.1 ~ 22.12.31 까지만!!

In [1]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from konlpy.corpus import kolaw
import nltk

**구축한 단어사전 불러오기**

In [4]:
word_dict = pd.read_csv('./data/단어합본.csv')
word_dict = word_dict.drop('Unnamed: 0', axis=1)
word_dict

,word,polarity
0,ㅡㅡ,-1
1,ㅠㅠ,-1
2,ㅠ_ㅠ,-1
3,ㅠ,-1
4,ㅜㅡ,-1
...,...,...
18408,대출,0
18409,횡령,-2
18410,축하,1
18411,행복,2


In [6]:
# 반복문에 돌리기 위해서 df를 list로 바꾸는 작업
word_list = word_dict.values.tolist()
word_list

[['ㅡㅡ', -1],
 ['ㅠㅠ', -1],
 ['ㅠ_ㅠ', -1],
 ['ㅠ', -1],
 ['ㅜㅡ', -1],
 ['ㅜㅜ', -1],
 ['ㅜ_ㅜ', -1],
 ['ㅜ.ㅜ', -1],
 ['ㅜ', -1],
 ['ㅗ', -1],
 ['힘찬 기운이', 2],
 ['힘찬', 2],
 ['힘차다', 2],
 ['힘차고 튼튼하다', 2],
 ['힘차고 대단한', 2],
 ['힘차고', 2],
 ['힘차게 진행되는', 2],
 ['힘차게 일어남', 1],
 ['힘차게', 2],
 ['힘이나 권세 위력 따위가 없이', -1],
 ['힘이 있다', 1],
 ['힘이 있고', 1],
 ['힘이 있게', 1],
 ['힘이 없음을', -1],
 ['힘이 없어서', -1],
 ['힘이 없어', -1],
 ['힘이 없다', -1],
 ['힘이 없는 사람을', -1],
 ['힘이 없는', -1],
 ['힘이 없거나', -1],
 ['힘이 약해진', -1],
 ['힘이 빠져', -1],
 ['힘이 뛰어나다', 1],
 ['힘이 들거나', -1],
 ['힘이 되어', 1],
 ['힘이 다 빠져', -1],
 ['힘이 기운차게', 2],
 ['힘이 강하다', 2],
 ['힘이 가득하고', 1],
 ['힘을 합하여', 1],
 ['힘을 잃고', -1],
 ['힘을 잃게', -1],
 ['힘을 잃거나', -1],
 ['힘을 못 쓰게', -1],
 ['힘을 들이고', -1],
 ['힘을 들였으나', -1],
 ['힘을 다함', 1],
 ['힘을 다하여', 1],
 ['힘을 다하다', 1],
 ['힘을 가지고', 1],
 ['힘에 부치는 데가 있다', -2],
 ['힘에 부쳐', -2],
 ['힘에 벅차', -2],
 ['힘에 눌려', -2],
 ['힘에 겹게', -2],
 ['힘에 겹거나', -2],
 ['힘에 겨워', -2],
 ['힘없이 흐느적흐느적', -1],
 ['힘없이 행동하다', -1],
 ['힘없이 행동하는', -1],
 ['힘없이 처질', -1],
 ['힘없이 주저앉는', -

### 다시 뽑은 뉴스 앞부분 -> 후반부에 붙이기

In [64]:
front_data = pd.read_csv('./data/front_sort.csv')
front_data = front_data.drop('Unnamed: 0', axis=1)
front_data

,제목,날짜
0,네이버·카카오 작년 연매출 '사상최대',2020.01.01.
1,한진·네이버·카카오·두산…국민연금 수익률 ‘효자’,2020.01.01.
2,[백브리핑] 돈보다 금배지?,2020.01.01.
3,게임위 'GO' 애플 'STOP'...19금 게임 이중검열 논란,2020.01.01.
4,달린 거리만큼 보험료 내는 ‘디지털 車보험’ 나온다,2020.01.01.
...,...,...
6460,NH농협은행 'NH직장인대출V' 출시 이벤트,2021.02.28.
6461,"""보너스로 1년 연봉 더 줄게""… '부르는 게 값' 된 개발자 영입 전쟁",2021.02.28.
6462,"크래프톤, 한번에 연봉 2000만원 인상…뉴웨이브 된 '성과 공유'",2021.02.28.
6463,"카카오, 5 대 1 비율로 액면분할",2021.02.28.


In [72]:
back_data = pd.read_csv('./data/all_news_data.csv', encoding= 'cp949')
back_data = back_data.drop('Unnamed: 0', axis=1)
back_data = back_data.drop('...1', axis=1)
back_data = back_data[8035:13034]
back_data

,제목,날짜
8035,"""사장 대신 팀장이 신속 결재""…카카오처럼 바뀌는 신한카드",2021.03.01.
8036,공동대표까지 나선 인사평가… 속 시끄러운 ‘IT 대표’ 카카오,2021.03.01.
8037,"[이코노 브리핑] 한화생명, 카카오페이와 업무협약 맺어 외",2021.03.01.
8038,"[이코노 브리핑] 한화생명, 카카오페이와 업무협약 맺어 외",2021.03.01.
8039,"[마켓트렌드]편의점, '이색 컬래버'로 화이트데이 공략",2021.03.01.
...,...,...
13029,"운수·화학·백화점, 코로나 2년 '반전스토리'",2021.12.31.
13030,'삼천피' 못지킨 2021 증시…그래도 '천스닥'은 꿋꿋,2021.12.31.
13031,견제장치 없어 ‘표적수사’ 폭주… 공수처 폐지론 확산,2021.12.31.
13032,연말 기업 신용등급 줄상향,2021.12.31.


In [73]:
### 위의 두 data 합치기
news_data =  pd.concat([front_data, back_data], ignore_index=True)
news_data

,제목,날짜
0,네이버·카카오 작년 연매출 '사상최대',2020.01.01.
1,한진·네이버·카카오·두산…국민연금 수익률 ‘효자’,2020.01.01.
2,[백브리핑] 돈보다 금배지?,2020.01.01.
3,게임위 'GO' 애플 'STOP'...19금 게임 이중검열 논란,2020.01.01.
4,달린 거리만큼 보험료 내는 ‘디지털 車보험’ 나온다,2020.01.01.
...,...,...
11459,"운수·화학·백화점, 코로나 2년 '반전스토리'",2021.12.31.
11460,'삼천피' 못지킨 2021 증시…그래도 '천스닥'은 꿋꿋,2021.12.31.
11461,견제장치 없어 ‘표적수사’ 폭주… 공수처 폐지론 확산,2021.12.31.
11462,연말 기업 신용등급 줄상향,2021.12.31.


In [76]:
# 중간저장
news_data.to_csv('./data/train_중간저장.csv')

### 합친 뉴스 파일을 토큰화 하는 작업

In [74]:
okt = Okt()

In [75]:
title = news_data['제목']

In [77]:
# 불용어 제거

for i in range(len(title)):
    title[i] = re.sub(r'[!"#$%&\'()*+,-./:·;<=>?@\[\]^_\`{|}~\\\\]','', title[i])
    
news_data['제목'] = title
news_data

,제목,날짜
0,네이버카카오 작년 연매출 사상최대,2020.01.01.
1,한진네이버카카오두산…국민연금 수익률 ‘효자’,2020.01.01.
2,백브리핑 돈보다 금배지,2020.01.01.
3,게임위 GO 애플 STOP19금 게임 이중검열 논란,2020.01.01.
4,달린 거리만큼 보험료 내는 ‘디지털 車보험’ 나온다,2020.01.01.
...,...,...
11459,운수화학백화점 코로나 2년 반전스토리,2021.12.31.
11460,삼천피 못지킨 2021 증시…그래도 천스닥은 꿋꿋,2021.12.31.
11461,견제장치 없어 ‘표적수사’ 폭주… 공수처 폐지론 확산,2021.12.31.
11462,연말 기업 신용등급 줄상향,2021.12.31.


In [78]:
okt_list = []

for i in range(len(title)):
    okt_list.append(okt.morphs(title[i]))
    
print(okt_list)

[['네이버', '카카오', '작년', '연매출', '사상', '최대'], ['한진', '네이버', '카카오', '두산', '…', '국민연금', '수익률', '‘', '효자', '’'], ['백', '브리핑', '돈', '보다', '금', '배지'], ['게임위', 'GO', '애플', 'STOP', '19', '금', '게임', '이중', '검열', '논란'], ['달린', '거리', '만큼', '보험료', '내는', '‘', '디지털', '車', '보험', '’', '나온다'], ['새', '로움', '보단', '익숙함', '…', '2020년', '주목', '할', '뮤지컬', '은'], ['새해', '자리', '바꾸는', 'IT', '리더', '들', '혁신', '으로', '경제', '살릴까'], ['내', '신', '성적', '영원불변', '…', '명문', '대', '수시', '합격', '비결', '은', '‘', '자율', '동아리', '’'], ['올해', '증시', '에서도', '믿을', '건', 'IT', '뿐'], ['민주당', '이낙연', '복귀', '초읽기', '…', '한국', '당', '보수', '통합', '올인'], ['네이버', '네이버', '페이', '투자', '유치', '로', '테크', '핀', '시장', '선점', '나선다'], ['트렌드', '쉐이크', '쉑', '도심', '속', '에서', '즐기는', '자연', '연출', '하다'], ['막막했던', '골목', '매장', '재', '창업', '패키지', '로', '숨통'], ['씨줄', '날줄', '간편', '후불', '결제', '장', '세훈', '논설', '위원'], ['네이버', '경', '영진', '베트남', '대학', '캠퍼스', '찾아간', '까닭', '은'], ['작년', '최', '다', '이용', '앱', '은', '카톡', '국민', '10', '명', '중', '7', '명', '이', '사용'], ['배달', '의', '민족', 'MA', '제동

In [79]:
news_data['token'] = okt_list
news_data = news_data[['날짜','제목','token']]
news_data

,날짜,제목,token
0,2020.01.01.,네이버카카오 작년 연매출 사상최대,"[네이버, 카카오, 작년, 연매출, 사상, 최대]"
1,2020.01.01.,한진네이버카카오두산…국민연금 수익률 ‘효자’,"[한진, 네이버, 카카오, 두산, …, 국민연금, 수익률, ‘, 효자, ’]"
2,2020.01.01.,백브리핑 돈보다 금배지,"[백, 브리핑, 돈, 보다, 금, 배지]"
3,2020.01.01.,게임위 GO 애플 STOP19금 게임 이중검열 논란,"[게임위, GO, 애플, STOP, 19, 금, 게임, 이중, 검열, 논란]"
4,2020.01.01.,달린 거리만큼 보험료 내는 ‘디지털 車보험’ 나온다,"[달린, 거리, 만큼, 보험료, 내는, ‘, 디지털, 車, 보험, ’, 나온다]"
...,...,...,...
11459,2021.12.31.,운수화학백화점 코로나 2년 반전스토리,"[운수, 화학, 백화점, 코로나, 2년, 반전, 스토리]"
11460,2021.12.31.,삼천피 못지킨 2021 증시…그래도 천스닥은 꿋꿋,"[삼, 천, 피, 못, 지킨, 2021, 증시, …, 그래도, 천스닥, 은, 꿋꿋]"
11461,2021.12.31.,견제장치 없어 ‘표적수사’ 폭주… 공수처 폐지론 확산,"[견제, 장치, 없어, ‘, 표적, 수사, ’, 폭주, …, 공, 수, 처, 폐지,..."
11462,2021.12.31.,연말 기업 신용등급 줄상향,"[연말, 기업, 신용등급, 줄, 상향]"


In [82]:
# 중간저장
news_data.to_csv('./data/train_token_중간저장.csv')

### 뉴스 토큰과 사전파일 대조하여 가중치 더하기

##### 일단 한 행에 먼저 테스트

In [83]:
word_list[0]

['ㅡㅡ', -1]

In [86]:
news_data['token'][3]

['게임위', 'GO', '애플', 'STOP', '19', '금', '게임', '이중', '검열', '논란']

In [100]:
count = 0

for i in word_list:
    for j in news_data['token'][3]:
        if j == i[0]:
            count += i[1]
            print(j)
            print(i[1])
print(count)

검열
-1
논란
-1
게임
0
애플
0
-2


##### 전체 적용 코드 생성

In [118]:
count = 0
a = []

for i in news_data['token']:
    li = []
    for j in i:
        for k in word_list:
            if j == k[0]:
                count += k[1]
                
    li.append(count)
    a.append(li)
    count = 0

a

[[1],
 [2],
 [1],
 [-2],
 [0],
 [1],
 [2],
 [1],
 [0],
 [-1],
 [4],
 [3],
 [1],
 [0],
 [0],
 [0],
 [-1],
 [-1],
 [-4],
 [-1],
 [0],
 [-3],
 [0],
 [-1],
 [0],
 [-1],
 [-1],
 [2],
 [2],
 [3],
 [-1],
 [0],
 [0],
 [-1],
 [0],
 [1],
 [-2],
 [1],
 [2],
 [-8],
 [0],
 [-2],
 [1],
 [1],
 [-2],
 [0],
 [2],
 [2],
 [1],
 [0],
 [2],
 [2],
 [0],
 [-13],
 [1],
 [6],
 [-4],
 [6],
 [1],
 [6],
 [0],
 [0],
 [0],
 [-5],
 [-2],
 [-1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [-2],
 [-2],
 [2],
 [2],
 [7],
 [0],
 [0],
 [0],
 [4],
 [-1],
 [-2],
 [-2],
 [-2],
 [0],
 [2],
 [-1],
 [1],
 [0],
 [0],
 [2],
 [1],
 [-4],
 [0],
 [0],
 [1],
 [0],
 [-3],
 [0],
 [-2],
 [-1],
 [1],
 [0],
 [-1],
 [-1],
 [4],
 [0],
 [0],
 [0],
 [-2],
 [-4],
 [-2],
 [1],
 [-6],
 [0],
 [0],
 [-1],
 [0],
 [-2],
 [1],
 [-1],
 [0],
 [-1],
 [1],
 [3],
 [-1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [-1],
 [1],
 [0],
 [-7],
 [-2],
 [0],
 [1],
 [0],
 [3],
 [0],
 [2],
 [-1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [8],
 [1],
 [0],
 [-1],
 [-1],
 [

In [120]:
# 길이 같음 -> 맞게 뽑힌 걸 알 수 있음
print(len(a))
print(len(news_data['token']))

11464
11464


In [146]:
## 긍정/부정/중립 분류하기 편하게 int 형식으로 바꿈

a_li = []

for i in a:    
    i = "".join([str(_) for _ in i])
    i = int(i)
    a_li.append(i)

len(a_li)

11464

In [147]:
news_data['가중치'] = a_li
news_data = news_data[['날짜','제목','token','가중치']]
news_data

,날짜,제목,token,가중치
0,2020.01.01.,네이버카카오 작년 연매출 사상최대,"[네이버, 카카오, 작년, 연매출, 사상, 최대]",1
1,2020.01.01.,한진네이버카카오두산…국민연금 수익률 ‘효자’,"[한진, 네이버, 카카오, 두산, …, 국민연금, 수익률, ‘, 효자, ’]",2
2,2020.01.01.,백브리핑 돈보다 금배지,"[백, 브리핑, 돈, 보다, 금, 배지]",1
3,2020.01.01.,게임위 GO 애플 STOP19금 게임 이중검열 논란,"[게임위, GO, 애플, STOP, 19, 금, 게임, 이중, 검열, 논란]",-2
4,2020.01.01.,달린 거리만큼 보험료 내는 ‘디지털 車보험’ 나온다,"[달린, 거리, 만큼, 보험료, 내는, ‘, 디지털, 車, 보험, ’, 나온다]",0
...,...,...,...,...
11459,2021.12.31.,운수화학백화점 코로나 2년 반전스토리,"[운수, 화학, 백화점, 코로나, 2년, 반전, 스토리]",-1
11460,2021.12.31.,삼천피 못지킨 2021 증시…그래도 천스닥은 꿋꿋,"[삼, 천, 피, 못, 지킨, 2021, 증시, …, 그래도, 천스닥, 은, 꿋꿋]",-1
11461,2021.12.31.,견제장치 없어 ‘표적수사’ 폭주… 공수처 폐지론 확산,"[견제, 장치, 없어, ‘, 표적, 수사, ’, 폭주, …, 공, 수, 처, 폐지,...",-3
11462,2021.12.31.,연말 기업 신용등급 줄상향,"[연말, 기업, 신용등급, 줄, 상향]",1


In [154]:
# 점수가 -1이하면 부정, 0이면 중립, 1이상이면 긍정으로 분류

sort_li = []

for i in news_data['가중치']:
    if i >= 1:
        sort_li.append('1')
    elif i == 0 :
        sort_li.append('0')
    else:
        sort_li.append('-1')

len(sort_li)

11464

In [160]:
news_data['분류'] = sort_li

In [163]:
# news_data = news_data[['날짜','제목','token','가중치','분류']]
# news_data = news_data.drop('token', axis=1)
# news_data = news_data.drop('분류', axis=1)
news_data.columns = ['date', 'text', 'score', 'senti']
news_data

,date,text,score,senti
0,2020.01.01.,네이버카카오 작년 연매출 사상최대,1,1
1,2020.01.01.,한진네이버카카오두산…국민연금 수익률 ‘효자’,2,1
2,2020.01.01.,백브리핑 돈보다 금배지,1,1
3,2020.01.01.,게임위 GO 애플 STOP19금 게임 이중검열 논란,-2,-1
4,2020.01.01.,달린 거리만큼 보험료 내는 ‘디지털 車보험’ 나온다,0,0
...,...,...,...,...
11459,2021.12.31.,운수화학백화점 코로나 2년 반전스토리,-1,-1
11460,2021.12.31.,삼천피 못지킨 2021 증시…그래도 천스닥은 꿋꿋,-1,-1
11461,2021.12.31.,견제장치 없어 ‘표적수사’ 폭주… 공수처 폐지론 확산,-3,-1
11462,2021.12.31.,연말 기업 신용등급 줄상향,1,1


In [164]:
# 최종 저장
news_data.to_csv('./data/긍부정분류_train.csv')